# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,18.07,85,12,2.01,AU,1738416541
1,1,adrar,20.5022,-10.0711,22.18,13,0,8.18,MR,1738416543
2,2,nemuro,43.3236,145.5750,-3.26,75,92,5.64,JP,1738416544
3,3,bilibino,68.0546,166.4372,-34.51,96,100,0.81,RU,1738416545
4,4,waitangi,-43.9535,-176.5597,16.01,94,100,1.79,NZ,1738416546


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
Humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
#    hover_cols = ["Address", "Bank Count"]
)

# Display the map
Humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [52]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[((city_data_df["Max Temp"] > 22) & (city_data_df["Max Temp"] < 26) & (city_data_df["Humidity"] <= 70) & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] < 50))]

# Drop any rows with null values
ideal_weather_df.dropna(how='any')

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
157,157,east london,-33.0153,27.9116,24.59,64,25,4.08,ZA,1738416701
199,199,al ghayzah,16.2079,52.1760,23.54,45,0,4.37,YE,1738416791
209,209,rio grande,-32.0350,-52.0986,25.35,63,0,0.86,BR,1738416803
283,283,khawr fakkan,25.3313,56.3420,23.03,40,0,4.13,AE,1738416898
291,291,bati,11.1833,40.0167,23.00,28,8,4.88,ET,1738416909
345,345,luwingu,-10.2621,29.9271,23.98,70,26,1.90,ZM,1738416857
431,431,diego de almagro,-26.3667,-70.0500,23.42,37,0,2.39,CL,1738417085
445,445,kaliganj,23.4095,89.1402,23.35,36,0,3.22,BD,1738417103
549,549,kampong thom,12.7111,104.8887,23.80,46,49,1.29,KH,1738417236
587,587,at taraf,25.3628,49.7250,23.30,18,0,1.17,SA,1738417286


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy(deep = True)
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
#hotel_df["Hotel Name"]
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
157,east london,ZA,-33.0153,27.9116,64,
199,al ghayzah,YE,16.2079,52.1760,45,
209,rio grande,BR,-32.0350,-52.0986,63,
283,khawr fakkan,AE,25.3313,56.3420,40,
291,bati,ET,11.1833,40.0167,28,
345,luwingu,ZM,-10.2621,29.9271,70,
431,diego de almagro,CL,-26.3667,-70.0500,37,
445,kaliganj,BD,23.4095,89.1402,36,
549,kampong thom,KH,12.7111,104.8887,46,
587,at taraf,SA,25.3628,49.7250,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 20000
limit = 1
categories = "accommodation.hotel"
params = {
    "categories":categories, 
    "limit":limit, 
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lng = row["Lng"]
    Lat = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}" 

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
east london - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
rio grande - nearest hotel: Hotel Vila Moura Executivo
khawr fakkan - nearest hotel: Raynor Hotel
bati - nearest hotel: No hotel found
luwingu - nearest hotel: No hotel found
diego de almagro - nearest hotel: Residencial Hambyll
kaliganj - nearest hotel: Molla Hotel মোল্লা হোটেল
kampong thom - nearest hotel: សណ្ឋាគារអរុណរះ
at taraf - nearest hotel: Grand Day Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
157,east london,ZA,-33.0153,27.9116,64,No hotel found
199,al ghayzah,YE,16.2079,52.1760,45,فندق تاج العرب
209,rio grande,BR,-32.0350,-52.0986,63,Hotel Vila Moura Executivo
283,khawr fakkan,AE,25.3313,56.3420,40,Raynor Hotel
291,bati,ET,11.1833,40.0167,28,No hotel found
345,luwingu,ZM,-10.2621,29.9271,70,No hotel found
431,diego de almagro,CL,-26.3667,-70.0500,37,Residencial Hambyll
445,kaliganj,BD,23.4095,89.1402,36,Molla Hotel মোল্লা হোটেল
549,kampong thom,KH,12.7111,104.8887,46,សណ្ឋាគារអរុណរះ
587,at taraf,SA,25.3628,49.7250,18,Grand Day Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
Vacation_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
Vacation_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)